In [1]:
import torch
from torch import empty, zeros
import math
import copy

In [2]:
torch.manual_seed(1)

In [3]:
class Module(object):
    def __init__(self):
        pass
        
    def forward(self, *input_):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []
    
    def update(self, lr):
        pass
    
    def zero_grad(self):
        pass
    
    def init_params(self, xavier_init, xavier_gain):
        pass


In [17]:
class Linear(Module):
    def __init__(self, N_in, N_out, xavier_init=False, xavier_gain=1):
        super(Linear, self).__init__()
        self.N_in = N_in
        self.N_out = N_out
        
        self.init_params(xavier_init, xavier_gain)
        
        self.gradW = zeros(self.W.shape)
        self.gradb = zeros(self.b.shape)
        
        self.m_weights = zeros(self.gradW.shape)
        self.m_bias = zeros(self.gradb.shape)
        
        self.v_weights = zeros(self.gradW.shape)
        self.v_bias = zeros(self.gradb.shape)
    
    def init_params(self, xavier_init=True, xavier_gain=1):
        if xavier_init:
            xavier_std = xavier_gain * math.sqrt(2.0 / (self.N_in + self.N_out))
        else:
            xavier_std = 1
            
        self.W = empty((self.N_in, self.N_out)).normal_(0, xavier_std)
        self.b = empty((1, self.N_out)).normal_(0, xavier_std)
    
    def forward(self, *input_):
        # out = W * input + b
        x = input_[0].clone()
        
        self.x = x
        
        return self.x.mm(self.W) + self.b
        
    def backward(self, *gradwrtoutput):
        # grad_w += input * x^(l-1).t()
        # grad_b += input
        # out = w.t() * input
        # input = grad of activation function, i.e. dl/ds^(l)
        # x^(l-1) = input of the forward pass
        input_ = gradwrtoutput[0].clone()
        
        self.gradW += self.x.t().mm(input_)
        self.gradb += input_.sum(0)
        
        return input_.mm(self.W.t())
        
    def param(self):
        return [(self.W, self.gradW, self.m_weights, self.v_weights), (self.b, self.gradb, self.m_bias, self.v_bias)]
    
    def update(self, lr):
        self.W.sub_(lr * self.gradW)
        self.b.sub_(lr * self.gradb)
    
    def zero_grad(self):
        self.gradW = zeros(self.W.shape)
        self.gradb = zeros(self.b.shape)

In [5]:
class ReLU(Module):
    def __init__(self, ):
        super(ReLU, self).__init__()
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        s[s < 0] = 0.
        
        return s
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out[out > 0] = 1
        out[out < 0] = 0
        
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [6]:
class LeakyReLU(Module):
    def __init__(self, alpha=0.01):
        super(LeakyReLU, self).__init__()
        self.alpha = alpha
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        s[s < 0] = self.alpha * s[s < 0]
        
        return s
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out[out > 0] = 1
        out[out < 0] = self.alpha
        
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [7]:
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        return s.tanh()
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out = 1 - out.tanh().pow(2)
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [8]:
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        return s.sigmoid()
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out = out.sigmoid() * (1 - out.sigmoid())
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [9]:
class Sequential(Module):
    def __init__(self, *modules, xavier_init=None, xavier_gain=1):
        super(Sequential, self).__init__()
        self.modules = modules
        self.xavier_init = xavier_init
        if xavier_init is not None:
            for module in self.modules:
                module.init_params(xavier_init, xavier_gain)
        
    def forward(self, *input_):
        x = input_[0].clone()
        
        for m in self.modules:
            x = m.forward(x)
        
        return x
        
    def backward(self, *gradwrtoutput):
        x = gradwrtoutput[0].clone()
        
        for i, m in enumerate(reversed(self.modules)):
            #print("{} : {}".format(i, x))
            x = m.backward(x)
        
    def param(self):
        params = []
        
        for m in self.modules:
            for param in m.param():
                params.append(param)
        
        return params

    def update(self, lr):
        for m in self.modules:
            m.update(lr)
    
    def zero_grad(self):
        for m in self.modules:
            m.zero_grad()

In [10]:
"""class Loss: 
    ...
"""
class LossMSE(Module):
    def __init__(self):
        super(LossMSE, self).__init__()
        pass
        
    def forward(self, y, target):
        # out = e^2
        # e = (y - f(x))
        self.y = y.clone()
        target_onehot = zeros((target.shape[0], 2)) 
        self.target = target_onehot.scatter_(1, target.view(-1, 1), 1)
        
        self.e = (self.y - self.target)
        self.n = self.y.size(0)
        
        
        return self.e.pow(2).sum()
        
    def backward(self):
        # out = 2 * e
        
        return 2 * self.e # / self.n        

In [11]:
class LossCrossEntropy(Module):
    def __init__(self):
        super(LossCrossEntropy, self).__init__()
        pass
        
    def forward(self, y, target):
        self.y = y.clone()
        self.target = target.clone()
        sm = torch.softmax(self.y, dim=1)
        likelihood = -torch.log(torch.clamp(sm[range(target.size(0)), target], min=1e-3, max=None))
        return likelihood.mean()
        
    def backward(self):
        sm = torch.softmax(self.y, dim=1)
        target_onehot = zeros((self.target.shape[0], 2)) 
        target_onehot = target_onehot.scatter_(1, self.target.view(-1, 1), 1)
        return sm - target_onehot

In [12]:
class Optimizer:
    def __init__(self):
        pass
    
    def train(self):
        raise NotImplementedError
    
    def step(self):
        raise NotImplementedError

class SGD(Optimizer):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-4, criterion=LossMSE()):
        super(SGD, self).__init__()
        self.model = model
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.mini_batch_size = mini_batch_size
        self.criterion = criterion
    
    def step(self):
        self.model.update(self.lr)
        
    # Train function?
    def train(self, train_input, train_target, verbose=True):
        
        for e in range(self.nb_epochs):
            sum_loss = 0.
        
            for b in range(0, train_input.size(0), self.mini_batch_size):
                self.model.zero_grad()
                output = self.model.forward(train_input.narrow(0, b, self.mini_batch_size))
                #print(output)
                loss = self.criterion.forward(output, train_target.narrow(0, b, self.mini_batch_size))
                # print(loss)
            
                sum_loss += loss
            
                l_grad = self.criterion.backward()
                self.model.backward(l_grad)
                #print(model.param()[0])
                self.step()
            
            #print(model.param()[0])
            if verbose:
                print("{} iteration: loss={}".format(e, sum_loss))
        return self.model

In [13]:
class Adam(Optimizer):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-3, criterion=LossMSE()):
        super(Adam, self).__init__()
        self.model = model
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.mini_batch_size = mini_batch_size
        self.criterion = criterion
        
        self.b1 = 0.9
        self.b2 = 0.999
        self.epsilon = 1e-8
        self.t = 0
    
    def step(self):
        self.t += 1
        
        for params in self.model.param():
            grad = params[1].clone()
            m = params[2]
            v = params[3]
            
            m = self.b1 * m + (1 - self.b1) * grad
            v = self.b2 * v + (1 - self.b2) * grad.pow(2)
            
            m_biasc = m / (1 - self.b1 ** self.t)
            v_biasc = v / (1 - self.b2 ** self.t)
            
            params[0].sub_(self.lr * m_biasc/(v_biasc.sqrt() + self.epsilon))
        
        
    # Train function?
    def train(self, train_input, train_target, verbose=True):
        
        for e in range(self.nb_epochs):
            sum_loss = 0.
        
            for b in range(0, train_input.size(0), self.mini_batch_size):
                self.model.zero_grad()
                output = self.model.forward(train_input.narrow(0, b, self.mini_batch_size))
                #print(output)
                loss = self.criterion.forward(output, train_target.narrow(0, b, self.mini_batch_size))
                # print(loss)
            
                sum_loss += loss
            
                l_grad = self.criterion.backward()
                self.model.backward(l_grad)
                #print(model.param()[0])
                self.step()
            
            #print(model.param()[0])
            if verbose:
                print("{} iteration: loss={}".format(e, sum_loss))
        return self.model

In [14]:
class Evaluator:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        
    def cross_validate(self, k=5, possible_lrs=[1e-5, 1e-4, 1e-3, 1e-2]):
        train_datasets = []
        test_datasets = []
        for i in range(k):
            train_datasets.append(generate_disc_set(1000))
            test_datasets.append(generate_disc_set(1000))
        scores = {}
        score_means = {}
        score_vars = {}
        for lr in possible_lrs:
            print("Validating:", lr)
            scores[lr] = []
            for (train_input, train_target), (test_input, test_target) in zip(train_datasets, test_datasets):
                optimizer.model = self.model 
                optimizer.lr = lr
                self.model = optimizer.train(train_input, train_target, verbose=False)
                accuracy = self.test(test_input, test_target)
                scores[lr].append(accuracy)
            scores[lr] = torch.FloatTensor(scores[lr])
            score_means[lr] = torch.mean(scores[lr])
            score_vars[lr] = torch.std(scores[lr])
            
        return score_means, score_vars
    
    def test(self, test_input, test_target):
        num_samples = test_input.size(0)
        prediction = self.model.forward(test_input)
        predicted_class = torch.argmax(prediction, axis=1)
        accuracy = sum(predicted_class == test_target).float() / num_samples
        return accuracy

In [15]:
def generate_disc_set(nb):
    input = empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

In [19]:
# model = Sequential(Linear(2, 25), ReLU(),
#                    Linear(25, 25), ReLU(),
#                    Linear(25, 25), ReLU(),
#                    Linear(25, 2))
# model = Sequential(Linear(2, 25), Tanh(),
#                    Linear(25, 25), Tanh(),
#                    Linear(25, 25), Tanh(),
#                    Linear(25, 2))
model = Sequential(Linear(2, 25), Sigmoid(),
                   Linear(25, 25), Sigmoid(),
                   Linear(25, 25), Sigmoid(),
                   Linear(25, 2))

optimizer = SGD(model, mini_batch_size=5)
evaluator = Evaluator(model, optimizer)

In [16]:
evaluator.cross_validate()

Validating: 1e-05
Validating: 0.0001
Validating: 0.001
Validating: 0.01


({1e-05: tensor(0.5828),
  0.0001: tensor(0.7502),
  0.001: tensor(0.9714),
  0.01: tensor(0.9654)},
 {1e-05: tensor(0.0706),
  0.0001: tensor(0.0697),
  0.001: tensor(0.0093),
  0.01: tensor(0.0234)})

In [22]:
best_lr = 1e-3
model = Sequential(Linear(2, 25), LeakyReLU(),
                   Linear(25, 25), LeakyReLU(),
                   Linear(25, 25), LeakyReLU(),
                   Linear(25, 2), xavier_init=True)
optimizer = Adam(model, lr=best_lr, mini_batch_size=100, criterion=LossMSE())
model = optimizer.train(train_input, train_target)
evaluator.model = model
evaluator.test(test_input, test_target)

0 iteration: loss=882.33056640625
1 iteration: loss=724.82177734375
2 iteration: loss=616.5814208984375
3 iteration: loss=540.6802368164062
4 iteration: loss=489.9297790527344
5 iteration: loss=458.3811950683594
6 iteration: loss=432.9735412597656
7 iteration: loss=407.1817626953125
8 iteration: loss=379.7580261230469
9 iteration: loss=350.23699951171875
10 iteration: loss=318.2288513183594
11 iteration: loss=284.6054382324219
12 iteration: loss=249.637451171875
13 iteration: loss=216.22174072265625
14 iteration: loss=187.2932891845703
15 iteration: loss=164.9610137939453
16 iteration: loss=149.39505004882812
17 iteration: loss=140.080322265625
18 iteration: loss=133.6435546875
19 iteration: loss=129.08587646484375
20 iteration: loss=125.69312286376953
21 iteration: loss=122.53553009033203
22 iteration: loss=119.55784606933594
23 iteration: loss=116.46913146972656
24 iteration: loss=113.84344482421875
25 iteration: loss=111.45858001708984
26 iteration: loss=108.47947692871094
27 iterat

tensor(0.9810)